# Blocking out the functionality for doc tech
This API will:
- receive an arbitrary query from the user as a textual transcript from speech to text
- will decide which of some number of fixed commands is relevent
- will return a structured response based on what is relevent

Some things this will do, for instance:
- text to page number
- text to command for scrolling
- text to which PDF to open

In [ ]:
!pip install langgraph langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 384.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OpenAIAPIKey')

# Note:
I'm thinking I'll first explore what type of thing I need to do

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#action determiner
class Action(TypedDict):
    scroll_up: bool
    scroll_down: bool
    next_page: bool
    previous_page: bool
    snap_page: bool
    find_fig: bool
    find_pdf: bool
    non_determ: bool

action_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Decide if the user wants one of the following actions performed:
            - `scroll_up`: scroll up a bit on the pdf
            - `scroll_down`: scroll down a bit on the pdf
            - `snap_page`: snap to a specific page of a pdf
            - `find_fig`: find a specific figure
            - `find_doc`: find a specific doc
            - `non_determ`: no valid action is discernable
            These are mutually exclusive. One should be true, the rest should be false.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

action_parser = action_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Action)

user_query = 'hey doctor show me a document with that one diagram about a horse'
action_parser.invoke({"messages": [("ai", "my name is doc tech, what action would you like me to perform?"),("user", user_query)]})

{'scroll_up': False,
 'scroll_down': False,
 'next_page': False,
 'previous_page': False,
 'snap_page': False,
 'find_fig': True,
 'find_pdf': False,
 'non_determ': False}

# Note:
ok now I need to handle each of these cases.

In [ ]:
class SnapPage(TypedDict):
    snap_page: int

snap_page_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Parse out the specific page of the pdf the user wants to snap to.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

snap_page_parser = snap_page_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(SnapPage)

user_query = 'go up two pages'
current_page = 6
snap_page_parser.invoke({"messages": [("ai", f"my name is doc tech, what page would you like to snap to. You are currently on page {current_page}"),("user", user_query)]})

{'snap_page': 8}

In [ ]:
class FigDesc(TypedDict):
    figure_description: str

fig_desc_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """The user wants to find a figure. Extract a description of the figure the user needs.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

fig_desc_parser = fig_desc_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(FigDesc)

user_query = 'hey doc tech, yeah, i uh, i need a diagram of a snow a uh a snow globe with no yea with out a top'
fig_desc_parser.invoke({"messages": [("ai", f"my name is doc tech, describe the figure you want me to find."),("user", user_query)]})

{'figure_description': 'a diagram of a snow globe without a top'}

In [ ]:
class DocDesc(TypedDict):
    doc_description: str

doc_desc_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """The user wants to find a document. Extract a description of the document the user needs.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

doc_desc_parser = doc_desc_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(DocDesc)

user_query = 'hey doc tech, yeah, i uh, i need a uh status report for twenty 24'
doc_desc_parser.invoke({"messages": [("ai", f"my name is doc tech, describe the figure you want me to find."),("user", user_query)]})

{'doc_description': 'status report for 2024'}

# Note:
I think it would be super useful to be able to understand if the user wants to be on the same pdf or not. So I'll make a parser for that general logic.

In [ ]:
class SameDoc(TypedDict):
    same_doc: bool

same_doc_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """if the user explicitly said they want to do something on this document.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

same_doc_parser = same_doc_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(SameDoc)

user_query = 'find that one colorful figure somewhere on this'
same_doc_parser.invoke({"messages": [("ai", f"my name is doc tech, describe a figure you want me to find."),("user", user_query)]})

{'same_doc': True}

# Note:
ok we're making headway. Now I need a way to search a GX bucket for some specific documents/figures

11795

In [ ]:
!pip install groundx-python-sdk --upgrade
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.4/359.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.1
    Uninstalling cryptography-43.0.1:
      Successfully uninstalled cryptography-43.0.1


In [ ]:
from pprint import pprint
from groundx import Groundx
from google.colab import userdata

groundx = Groundx(
    api_key=userdata.get('GroundXAPIKey_daniel.warfield')
)

In [ ]:
from pprint import pprint
from groundx import Groundx

bucket_id = 11795

def gx_search_figure(query):

    response = groundx.search.content(
        id=bucket_id,
        query=query
    )

    semantic_object = response.body['search']['results'][0]

    return semantic_object['sourceUrl'], semantic_object['boundingBoxes'][0]['pageNumber']

gx_search_figure('Theres a colorful block puzzle thing and I cant figure it out')

('https://upload.groundx.ai/file/11795/babys-first-blocks.pdf', 2)

In [ ]:
from pprint import pprint
from groundx import Groundx

bucket_id = 11795

def gx_search_document(query):

    response = groundx.search.content(
        id=bucket_id,
        query=query
    )

    semantic_object = response.body['search']['results'][0]

    return semantic_object['sourceUrl']

gx_search_document('build document of a coffee table')

'https://upload.groundx.ai/file/11795/lack-coffee-table-black-brown__aa-472482-3-2.pdf'

# Tying it all together
given a query:
- decide on the action to take
- do any processing that's required
- return a response

In [ ]:
!pip install langgraph langchain-openai
!pip install groundx-python-sdk --upgrade
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import userdata
os.environ['GROUNDX_API_KEY'] = userdata.get('GroundXAPIKey_daniel.warfield')
os.environ['OPENAI_API_KEY'] = userdata.get('OpenAIAPIKey')

In [ ]:
from groundx import Groundx
from google.colab import userdata
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os

# Setting up API keys
groundx = Groundx(
    api_key=os.environ['GROUNDX_API_KEY']
)

#===============================================================================
# Action Parsing
#===============================================================================

#action determiner
class Action(TypedDict):
    scroll_up: bool
    scroll_down: bool
    next_page: bool
    previous_page: bool
    snap_page: bool
    find_fig: bool
    find_pdf: bool
    non_determ: bool

action_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Decide if the user wants one of the following actions performed:
            - `scroll_up`: scroll up a small amount within one page of the pdf
            - `scroll_down`: scroll down a small amount within one page of the pdf
            - `snap_page`: snap to a specific page of a pdf
            - `find_fig`: find a specific figure, table, image, or specific item.
            - `find_doc`: find a specific doc
            - `non_determ`: no valid action is discernable
            These are mutually exclusive. One should be true, the rest should be false.
            note: you can use snap_page to go to a page relative to the current page.
            note: blanket questions should default to find figure, unless they're obviously about a document
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

action_parser = action_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Action)

#===============================================================================
# Snap Page Parsing
#===============================================================================

class SnapPage(TypedDict):
    snap_page: int

snap_page_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Parse out the specific page of the pdf the user wants to snap to.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

snap_page_parser = snap_page_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(SnapPage)

#===============================================================================
# Figure Description Parsing
#===============================================================================

class FigDesc(TypedDict):
    figure_description: str

fig_desc_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """The user wants to find a figure. Extract a description of the figure the user needs.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

fig_desc_parser = fig_desc_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(FigDesc)

#===============================================================================
# Document Description Parsing
#===============================================================================

class DocDesc(TypedDict):
    doc_description: str

doc_desc_parse_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """The user wants to find a document. Extract a description of the document the user needs.
            """,
        ),
        ("placeholder", "{messages}"),
    ]
)

doc_desc_parser = doc_desc_parse_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(DocDesc)

user_query = 'hey doc tech, yeah, i uh, i need a uh status report for twenty 24'
doc_desc_parser.invoke({"messages": [("ai", f"my name is doc tech, describe the figure you want me to find."),("user", user_query)]})

#===============================================================================
# Search for Figure
#===============================================================================

bucket_id = 11795

def gx_search_figure(query):

    response = groundx.search.content(
        id=bucket_id,
        query=query
    )

    semantic_object = response.body['search']['results'][0]

    return semantic_object['sourceUrl'], semantic_object['boundingBoxes'][0]['pageNumber']

#===============================================================================
# Search for Documents
#===============================================================================

def gx_search_document(query):

    response = groundx.search.content(
        id=bucket_id,
        query=query
    )

    semantic_object = response.body['search']['results'][0]

    return semantic_object['sourceUrl']

#===============================================================================
# Endpoint
#===============================================================================

def handle_query(query, context):
    #getting action that should be performed
    response = action_parser.invoke({"messages": [("ai", "my name is doc tech, what action would you like me to perform?"),("user", query)]})
    response['pdf']= None
    response['page']=None

    #doing follow up as necessary
    if response['snap_page']:
        response['page']=snap_page_parser.invoke({"messages": [("ai", f"my name is doc tech, what page would you like to snap to. Current state: {context}"),("user", query)]})
    elif response['find_fig']:
        response['pdf'], response['page'] = gx_search_figure(query)
    elif response['find_pdf']:
        response['pdf'] = gx_search_document(query)

    return response

In [ ]:
handle_query('Show me the portfolio overlap of the s&p 500', {'current_page': 6})

{'scroll_up': False,
 'scroll_down': False,
 'next_page': False,
 'previous_page': False,
 'snap_page': False,
 'find_fig': True,
 'find_pdf': False,
 'non_determ': False,
 'pdf': 'https://upload.groundx.ai/file/11795/dashboard-sp-500-factor.pdf',
 'page': 5}

In [ ]:
handle_query('Theres a colorful block puzzle thing and I cant figure it out?', {'current_page': 1})

{'scroll_up': False,
 'scroll_down': False,
 'snap_page': False,
 'find_fig': True,
 'find_pdf': False,
 'non_determ': False,
 'pdf': 'https://upload.groundx.ai/file/11795/babys-first-blocks.pdf',
 'page': 2}

In [ ]:
handle_query('Scroll up', {'current_page': 1})

{'scroll_up': True,
 'scroll_down': False,
 'snap_page': False,
 'find_fig': False,
 'find_pdf': False,
 'non_determ': False,
 'pdf': None,
 'page': None}

In [ ]:
handle_query('Pull up the S and P five hundred document', {'current_page': 1})

{'scroll_up': False,
 'scroll_down': False,
 'snap_page': False,
 'find_fig': False,
 'find_pdf': True,
 'non_determ': False,
 'pdf': 'https://upload.groundx.ai/file/11795/dashboard-sp-500-factor.pdf',
 'page': None}

In [ ]:
handle_query('How do I attatch the shelf on this table?', {'current_page': 1})

{'scroll_up': False,
 'scroll_down': False,
 'snap_page': False,
 'find_fig': True,
 'find_pdf': False,
 'non_determ': False,
 'pdf': 'https://upload.groundx.ai/file/11795/lack-coffee-table-black-brown__aa-472482-3-2.pdf',
 'page': 7}